# Create Kd vectors for BA1 antibodies, using WT as a background


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
BEGIN=18
END=5
BA1="NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKST"
BA1=BA1[BEGIN:-END]
len(BA1)

178

In [2]:
# import matplotlib.pyplot as plt#5,7 best:.42, .47; pnas: 18,5

import os, sys
pgm_path = '../PGM/'

sys.path.append(pgm_path + 'source/')
sys.path.append(pgm_path + 'utilities/')
# sys.path.append('DNA_utils/')
# imports
import utilities, Proteins_utils, sequence_logo, plots_utils 
from sklearn.linear_model import Ridge

sys.path.append('../')
from utils import *

e:\ESCAPE_MAP_DRAFT\exp_data\../PGM/source\numba_utilities.py:1124: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float32, 2, 'F', False, aligned=True), Array(float32, 2, 'A', False, aligned=True))
  dmean_v_dw = np.dot(s1.T, V)
e:\ESCAPE_MAP_DRAFT\exp_data\../PGM/source\numba_utilities.py:961: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float32, 1, 'A', False, aligned=True), Array(float32, 2, 'A', False, aligned=True))
  mean_V = np.dot(weights, V) / sum_weights
C:\Users\maria\AppData\Roaming\Python\Python312\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [3]:
ic50_db=pd.read_csv('../exp_data/cao_data/antibody_ic50s.csv')
ic50_db=ic50_db[ic50_db['virus']=='D614G']

source_db=pd.read_csv('cao_data/antibody_sources.csv')
escape_db=pd.read_csv('cao_data/escape.csv')
# rows with source == "WT convalescents" (case/space tolerant)
wts = source_db[source_db["source"].astype(str) == "BA.1 convalescents"]
out = wts[["antibody", "source", "study"]].dropna(subset=["antibody"]).drop_duplicates()
conditions = sorted(out["antibody"].unique().tolist())

#get antibody_list that are in ic50_db
conditions = [ab for ab in conditions if ab in ic50_db['antibody'].values]
print(len(conditions))

671


C:\Users\maria\AppData\Local\Temp\ipykernel_31752\2370534369.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  escape_db=pd.read_csv('cao_data/escape.csv')


In [4]:
escape_db=escape_db[escape_db['site']>=349] 
escape_db=escape_db[escape_db['site']<=526]
escape_db

,site,wildtype,mutation,mut_escape,antibody,group,study
58,365,Y,A,0.50810,BD55-5840,D2.440,repeated_2023
59,365,Y,D,0.49170,BD55-5840,D2.440,repeated_2023
60,365,Y,F,0.10140,BD55-5840,D2.440,repeated_2023
61,365,Y,G,0.46810,BD55-5840,D2.440,repeated_2023
62,365,Y,I,0.12810,BD55-5840,D2.440,repeated_2023
...,...,...,...,...,...,...,...
577098,514,NaN,F,0.01613,XGv-422,E3,imprinted_2022
577099,514,NaN,I,0.02397,XGv-422,E3,imprinted_2022
577100,514,NaN,M,0.05784,XGv-422,E3,imprinted_2022
577101,514,NaN,R,0.02008,XGv-422,E3,imprinted_2022


In [5]:
def get_sequence(WT, site,	wildtype_mut,	mutation):
    #site is expected to be 349 instead of 0
    site=site-349
    return WT[:site]+mutation+WT[site+1:]

#example:
print(get_sequence(BA1, 349, 'S', 'A'))

escape_db['sequence']=escape_db.apply(lambda x: get_sequence(BA1, x['site'], x['wildtype'], x['mutation']), axis=1)
    

AVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCG


In [6]:
# escape_db['mut_escape']: put a floor of 0.005
escape_db['mut_escape']=escape_db['mut_escape'].apply(lambda x: max(x,0.01))
#create log mut escape
escape_db['log_mut_escape']=np.log(escape_db['mut_escape'])


# Normalized K_d vectors so that logKd (wt)=IC50

### create KD

In [7]:
BA1_SEQ=Proteins_utils.seq2num(BA1)[0]
#betwee 18 and 5
BA1_SEQ.shape


(178,)

In [8]:

ONE_HOT_BA1=np.array([BA1_SEQ])
ONE_HOT_BA1=one_hot_encoding(ONE_HOT_BA1)

ONE_HOT_BA1.shape

(1, 3738)

In [9]:
# count nb of zeros
nb_zeros = np.sum(ONE_HOT_BA1 == 0)
print('Number of zeros:', nb_zeros)
# count nb of ones	
nb_ones = np.sum(ONE_HOT_BA1 == 1)
print('Number of ones:', nb_ones)


Number of zeros: 3560
Number of ones: 178


In [10]:

from sklearn.linear_model import Ridge
ab_names=[]
ab_ic50_array=[]
for condition in conditions:
    print('condition',condition)
    df_condition=escape_db[escape_db['antibody']==condition]
    print('found', df_condition.shape[0], 'mutations')

    #find min non 0 value of mut_escape
    #replace 0 with min_value

    #create log mut escape
    df_condition['delta_G']=np.log(df_condition['mut_escape'])

    #in df_condition['log_mut_escape'], replace -inf with min value -1
    index_removed=[]
    with open('fasta_files/'+condition+'deltaG.fasta', 'w') as f:
        for index, row in df_condition.iterrows():
            #if * in sequence, remove sequence
            if '*' in row['sequence']:
                index_removed.append(index)
                continue
            f.write('>' + str(index) + '\n')
            f.write(row['sequence'] + '\n')

    condition_sequences=Proteins_utils.load_FASTA('fasta_files/'+condition+'deltaG.fasta', drop_duplicates=False)
    #to array
    condition_sequences=np.array(condition_sequences)

    #shape
    one_hot=one_hot_encoding(condition_sequences)


    X_condition=np.concatenate([one_hot, ONE_HOT_BA1], axis=0)
    
    y=df_condition['delta_G'].values
    #remove index_removed from y
    y=np.delete(y, index_removed)

    wt_deltaG=df_condition['delta_G'].min()
    #assert wt_deltaG is not nan or zero or inf
    assert not np.isnan(wt_deltaG)
    assert wt_deltaG!=0
    assert not np.isinf(wt_deltaG)

    
    y=np.append(y,wt_deltaG)



    lr=Ridge(alpha=0.1)
    lr.fit(X_condition, y)
    #intersection
    intersect_0=lr.intercept_

    #compute new intercept so that WT has -23 delta_G
    #compute delta_G of WT
    delta_G_WT_0=lr.predict(ONE_HOT_BA1)[0]

    

    #new intercept
    try:
        wt_ic50=ic50_db[ic50_db['antibody']==condition]['IC50'].values[0]
        #str to int
        wt_ic50=float(wt_ic50)*0.000001
        print('wt_ic50', wt_ic50)
    except:
        print('antibody', condition, 'not found in ic50_db')
        raise Exception('antibody not found')
    
    ab_names.append(condition)
    ab_ic50_array.append(wt_ic50)

    new_intercept=intersect_0-delta_G_WT_0+np.log(wt_ic50)



    q=lr.coef_
    #make sure no nan or inf coeff
    assert not np.isnan(q).any()
    assert not np.isinf(q).any()

    #compute delta_G of WT using scalar product with q
    delta_G_WT=np.dot(q, ONE_HOT_BA1[0])+new_intercept
    print('delta_G_WT new', delta_G_WT, 'real delta_G_WT', np.log(wt_ic50))
    #add intercept to the end of q
    q=np.append(q, new_intercept)
    #score
    print('score', lr.score(one_hot, df_condition['delta_G']))


        # q=covariance_vector(one_hot, df_condition['mut_escape'])
    print(q.shape)
    #save in q_vectors folder
    np.save('ba1_kd_vectors/'+condition+'delta_G.npy', q)
print('all conditions done')

condition BD56-001
found 147 mutations


wt_ic50 1.93e-08
delta_G_WT new -17.76316074103557 real delta_G_WT -17.76316074103557
score 0.9936194725161454
(3739,)
condition BD56-003
found 196 mutations


C:\Users\maria\AppData\Local\Temp\ipykernel_31752\3080826001.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_condition['delta_G']=np.log(df_condition['mut_escape'])
C:\Users\maria\AppData\Local\Temp\ipykernel_31752\3080826001.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_condition['delta_G']=np.log(df_condition['mut_escape'])


wt_ic50 4.81e-09
delta_G_WT new -19.152568752828742 real delta_G_WT -19.152568752828742
score 0.9974749327598882
(3739,)
condition BD56-004
found 274 mutations
wt_ic50 1.54e-08


C:\Users\maria\AppData\Local\Temp\ipykernel_31752\3080826001.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_condition['delta_G']=np.log(df_condition['mut_escape'])
C:\Users\maria\AppData\Local\Temp\ipykernel_31752\3080826001.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_condition['delta_G']=np.log(df_condition['mut_escape'])


...

C:\Users\maria\AppData\Local\Temp\ipykernel_31752\3080826001.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_condition['delta_G']=np.log(df_condition['mut_escape'])
C:\Users\maria\AppData\Local\Temp\ipykernel_31752\3080826001.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_condition['delta_G']=np.log(df_condition['mut_escape'])


wt_ic50 9.999999999999999e-06
delta_G_WT new -11.51292546497023 real delta_G_WT -11.512925464970229
score 0.9805130106579375
(3739,)
condition SN1047
found 279 mutations
wt_ic50 9.999999999999999e-06
delta_G_WT new -11.512925464970229 real delta_G_WT -11.512925464970229
score 0.9795489708703897
(3739,)
condition SN1104


C:\Users\maria\AppData\Local\Temp\ipykernel_31752\3080826001.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_condition['delta_G']=np.log(df_condition['mut_escape'])
C:\Users\maria\AppData\Local\Temp\ipykernel_31752\3080826001.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_condition['delta_G']=np.log(df_condition['mut_escape'])


found 249 mutations
wt_ic50 9.999999999999999e-06
delta_G_WT new -11.512925464970229 real delta_G_WT -11.512925464970229
score 0.9847051669137871
(3739,)
condition SN1138
found 109 mutations
wt_ic50 9.999999999999999e-06
delta_G_WT new -11.512925464970227 real delta_G_WT -11.512925464970229
score 0.9573214831831423
(3739,)
all conditions done


C:\Users\maria\AppData\Local\Temp\ipykernel_31752\3080826001.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_condition['delta_G']=np.log(df_condition['mut_escape'])
